In [1]:
import sys
import importlib.util

%pip install -q "openvino>=2023.1.0" "nncf==2.4.0" "tensorflow-hub>=0.15.0" "tensorflow_datasets"


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os, sys
import numpy as np
from pathlib import Path 

from openvino.runtime import Core
import openvino as ov
import nncf
import logging

sys.path.append("../utils")
from notebook_utils import download_file
from nncf.common.logging.logger import set_log_level
set_log_level(logging.ERROR)

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow import keras

from PIL import Image
from sklearn.metrics import accuracy_score

2023-12-11 15:58:57.883729: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-11 15:58:57.886838: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 15:58:57.929086: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 15:58:57.929118: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 15:58:57.929146: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

[ WARNING ] /home/psakhamo/miniconda3/envs/big_transfer_ipynb/lib/python3.11/site-packages/openvino/tools/accuracy_checker/preprocessor/launcher_preprocessing/ie_preprocessor.py:21: FutureWarning: OpenVINO Inference Engine Python API is deprecated and will be removed in 2024.0 release.For instructions on transitioning to the new API, please refer to https://docs.openvino.ai/latest/openvino_2_0_transition_guide.html
  from openvino.inference_engine import ResizeAlgorithm, PreProcessInfo, ColorFormat, MeanVariant  # pylint: disable=import-outside-toplevel,package-absolute-imports

[ WARNING ] /home/psakhamo/miniconda3/envs/big_transfer_ipynb/lib/python3.11/site-packages/openvino/tools/accuracy_checker/launcher/dlsdk_launcher.py:60: FutureWarning: OpenVINO nGraph Python API is deprecated and will be removed in 2024.0 release.For instructions on transitioning to the new API, please refer to https://docs.openvino.ai/latest/openvino_2_0_transition_guide.html
  import ngraph as ng



Post-training Optimization Tool is deprecated and will be removed in the future. Please use Neural Network Compression Framework instead: https://github.com/openvinotoolkit/nncf
Nevergrad package could not be imported. If you are planning to use any hyperparameter optimization algo, consider installing it using pip. This implies advanced usage of the tool. Note that nevergrad is compatible only with Python 3.8+


INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, onnx, openvino


Initializations

In [3]:
ie = Core()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


# For top 5 labels.
MAX_PREDS = 1
TRAINING_BATCH_SIZE = 128
BATCH_SIZE = 1
IMG_SIZE = (256, 256)  # Default Imagenet image size
NUM_CLASSES = 10  # For Imagenette dataset
FINE_TUNING_STEPS = 1
LR = 1e-5

MEAN_RGB = (0.485 * 255, 0.456 * 255, 0.406 * 255)  # From Imagenet dataset
STDDEV_RGB = (0.229 * 255, 0.224 * 255, 0.225 * 255)  # From Imagenet dataset


Dataset Preprocessing

In [4]:
datasets, datasets_info = tfds.load('imagenette/160px', shuffle_files=True, as_supervised=True, with_info=True,
                                    read_config=tfds.ReadConfig(shuffle_seed=0))
train_ds, validation_ds = datasets['train'], datasets['validation']
#ds, ds_info = tfds.load('imagenette/full-size', split='validation', shuffle_files=True, as_supervised=True, with_info=True,
#                                    read_config=tfds.ReadConfig(shuffle_seed=0))
#train_dataset, validation_dataset = datasets['train'], datasets['validation']
#validation_dataset = ds

In [5]:

def preprocessing(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label


train_dataset = (train_ds.map(preprocessing, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                              .batch(TRAINING_BATCH_SIZE)
                              .prefetch(tf.data.experimental.AUTOTUNE))

validation_dataset = (validation_ds.map(preprocessing, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                                        .batch(TRAINING_BATCH_SIZE)
                                        .prefetch(tf.data.experimental.AUTOTUNE))

Tensorflow BiT Classification model fine-tuning

In [6]:
# Load the BiT model
bit_model_url = "https://tfhub.dev/google/bit/m-r50x1/1"
bit_m = hub.KerasLayer(bit_model_url, trainable=True)

# Customize the model for the new task
model = tf.keras.Sequential([
    bit_m,
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
model.fit(train_dataset.take(5000),
          epochs=FINE_TUNING_STEPS,
          validation_data=validation_dataset.take(2000))
model.save("./bit_tf_model/", save_format='tf')

101/101 [==============================] - 479s 4s/step - loss: 0.5609 - accuracy: 0.8675 - val_loss: 0.0756 - val_accuracy: 0.9780


OpenVINO Model Optimization: TF FP32 to OV FP32

In [7]:
ir_path = Path("./bit_ov_model/bit_m_r50x1_1.xml")
if not ir_path.exists():
    print("Exporting Tensorflow model to IR..!!!")
    ov_model = ov.convert_model("./bit_tf_model")
    ov.save_model(ov_model, ir_path)
else:
    print(f"IR model {ir_path} already exists.")
   

Exporting Tensorflow model to IR..!!!


TF FP32 Model Accuracy

In [8]:
tf_model = tf.keras.models.load_model("./bit_tf_model/")
   
tf_predictions = []
gt_label = []

for _, label in validation_dataset:
    for l in label:
        l_list = l.numpy().tolist()
        gt_label.append(l_list.index(1))
        
for img_batch, label_batch in validation_dataset:
    tf_result_batch = tf_model.predict(img_batch, verbose=0)
    for i in range(len(img_batch)):
        tf_result = tf_result_batch[i]
        tf_result = tf.reshape(tf_result, [-1])
        top5_label_idx = np.argsort(tf_result)[-MAX_PREDS::][::-1]
        tf_predictions.append(top5_label_idx)

# Convert the lists to NumPy arrays for accuracy calculation
tf_predictions = np.array(tf_predictions)
gt_label = np.array(gt_label)

acc_score = accuracy_score(tf_predictions, gt_label)
print(f"TF FP32 Model Accuracy = {acc_score}")

2023-12-11 16:09:21.910833: W tensorflow/core/common_runtime/graph_constructor.cc:839] Node 're_lu_48/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 2 outputs. Output shapes may be inaccurate.
2023-12-11 16:09:21.910906: W tensorflow/core/common_runtime/graph_constructor.cc:839] Node 'global_average_pooling2d/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.


TF FP32 Model Accuracy = 0.978


BiT OpenVINO FP32 Model Accuracy 

In [9]:
ov_fp32_model = ie.read_model("./bit_ov_model/bit_m_r50x1_1.xml")
ov_fp32_model.reshape([1, IMG_SIZE[0], IMG_SIZE[1], 3])

# Target device set to CPU (Other options Ex: AUTO/GPU/dGPU/)
compiled_model = ov.compile_model(ov_fp32_model)#, device_name="CPU")
output = compiled_model.outputs[0]

ov_predictions = []
for img_batch, _ in validation_dataset:
    for image in img_batch:
        image = tf.expand_dims(image, axis=0)
        pred = compiled_model(image)[output]
        ov_result = tf.reshape(pred, [-1])
        top_label_idx = np.argsort(ov_result)[-MAX_PREDS::][::-1]
        ov_predictions.append(top_label_idx)

acc_score = accuracy_score(ov_predictions, gt_label)
print(f"OV FP32 Model Accuracy = {acc_score}")

OV FP32 Model Accuracy = 0.978


NNCF: BiT OpenVINO FP32 Model Quantization to INT8

In [10]:
def nncf_preprocessing(image, label):
  image = tf.image.resize(image, IMG_SIZE)
  image = image - MEAN_RGB
  image = image / STDDEV_RGB
  label = tf.one_hot(label, NUM_CLASSES)
  return image#, label

val_ds = (validation_ds.map(nncf_preprocessing, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                                        .batch(1)
                                        .prefetch(tf.data.experimental.AUTOTUNE))

print("Preparing NNCF Calibration dataset....!!!")
calibration_dataset = nncf.Dataset(val_ds)
    
ov_fp32_model = ie.read_model("./bit_ov_model/bit_m_r50x1_1.xml")
#ov_fp32_model.reshape([1, IMG_SIZE[0], IMG_SIZE[1], 3])
    
#print(f"ov_fp32_model.input_info = {ov_fp32_model.input_info}")
print("Started NNCF qunatization process")
ov_int8_model = nncf.quantize(ov_fp32_model, calibration_dataset, fast_bias_correction=False)

ov.serialize(ov_int8_model, "./bit_ov_int8_model/bit_m_r50x1_1_ov_int8.xml")
print("NNCF qunatization process completed..!!!")

Preparing NNCF Calibration dataset....!!!
Started NNCF qunatization process
NNCF qunatization process completed..!!!


BiT OpenVINO INT8 Model Accuracy

In [11]:
nncf_quantized_model = ie.read_model("./bit_ov_int8_model/bit_m_r50x1_1_ov_int8.xml")
nncf_quantized_model.reshape([1, IMG_SIZE[0], IMG_SIZE[1], 3])

In [12]:
# Release previously compiled fp32 OV model
#compiled_model.release()
# Target device set to CPU (Other options Ex: AUTO/GPU/dGPU/)
compiled_model = ov.compile_model(nncf_quantized_model)
output = compiled_model.outputs[0]

In [13]:
ov_predictions = []
inp_tensor = nncf_quantized_model.inputs[0]
out_tensor = nncf_quantized_model.outputs[0]
        
for img_batch, _ in validation_dataset:
    for image in img_batch:
        image = tf.expand_dims(image, axis=0)
        pred = compiled_model(image)[output]
        ov_result = tf.reshape(pred, [-1])
        top_label_idx = np.argsort(ov_result)[-MAX_PREDS::][::-1]
        ov_predictions.append(top_label_idx)
        
acc_score = accuracy_score(ov_predictions, gt_label)
print(f"OV INT8 Model Accuracy = {acc_score}")

OV INT8 Model Accuracy = 0.974
